# 🗺️ PDF 旅遊行程轉換工具

本 Notebook 將協助您：
1. 將 PDF 旅遊行程文件轉換為 Markdown 格式（供 Claude 閱讀）
2. 產生 Lovart.ai 圖片生成提示詞
3. 建立可對外分享的互動式網頁

---

## 📦 1. 安裝與匯入必要套件

In [ ]:
# 安裝必要套件
!pip install pymupdf -q

In [1]:
import fitz  # PyMuPDF
import os
import json
import re
from datetime import datetime

print("✅ 套件匯入成功！")

✅ 套件匯入成功！


---

## 📖 2. PDF 讀取與文字萃取

讀取 `inputs/哈爾濱.pdf` 檔案並提取所有文字內容。

In [2]:
# 設定 PDF 檔案路徑
pdf_path = r"d:\Work\TestProject\產生旅行行程\inputs\哈爾濱.pdf"

# 開啟 PDF 並提取文字
doc = fitz.open(pdf_path)

print(f"📄 PDF 檔案: {os.path.basename(pdf_path)}")
print(f"📃 總頁數: {len(doc)} 頁")
print()

# 提取每頁文字
all_text = []
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text = page.get_text()
    all_text.append({
        "page": page_num + 1,
        "content": text
    })
    
# 顯示前幾頁內容預覽
print("=" * 60)
print("📝 內容預覽 (前 3 頁)")
print("=" * 60)
for page_data in all_text[:3]:
    print(f"\n【第 {page_data['page']} 頁】")
    print("-" * 40)
    print(page_data['content'][:1000])
    print("...")

doc.close()

📄 PDF 檔案: 哈爾濱.pdf
📃 總頁數: 17 頁

📝 內容預覽 (前 3 頁)

【第 1 頁】
----------------------------------------
團號：KRB011009CA6 東北冰雪奇緣～雪鄉蘑菇屋、冰雪大世界、魔界長白山、三晚五
星飯店九日  
含稅  無自費  無購物站  贈參團好禮   
2026/01/10 台北出發 9 天｜訂金 15,000 /人 
★ 哈爾濱國際冰雪節、冰雪大世界、雪鄉蘑菇屋 
★ 魔界長白山(含霧淞飄流)、俄式風情伏爾加莊園 
★ 季節限定：香格里拉冰宮火鍋、三晚品牌五星酒店 
$48,500  
主要景點  
 
行程含27 個主要景點 
D1 桃園／北京／長春  
D2 長春 > 淨月潭森林公園(觀藍冰) > 地質宮外觀 > 長春文化廣場 > (車程約1.5HR) > 吉林 > 車遊臨江門大橋 > 天主教堂
廣場 > 吉林  
D3 吉林 > 霧淞長廊 > (車程約4HR) > 長白山 > 長白山魔界景區(含霧淞漂流) > 長白山  
D4 長白山 > 長白山北坡風景區(含環保車及倒站車)、天池、小天池、溫泉群、瀑布、綠淵潭 > (車程約2.5HR) > 敦化  
D5 敦化 > (車程約4.5HR) > 鏡泊湖 > 鏡泊湖吊水樓冰瀑 > 鏡泊湖觀冬捕 > (車程約2HR) > 雪鄉 > 雪鄉夜景 > 雪鄉  
D6 雪鄉 > 網紅打卡-雪韻大街 > 觀光棧道 > 雪鄉郵局 > 民俗博物館 > (車程約5H) > 伏爾加莊園  
D7 伏爾加莊園(含高山雪圈每人一次) > 伏特加酒堡(品嚐伏特加酒+小點) > 彼得洛夫藝術宮 > (車程約1HR) > 哈爾濱 > 百
年濱洲鐵路橋 > 音樂公園網紅大雪人 > 中央大街 > 聖索菲亞教堂廣場 > 冰雪大世界 > (文化交流參訪) > 哈爾濱  
D8 哈爾濱／上海 > EKA 天物 > 外觀浦東三大高樓 > 上海  
D9 上海 > 新場古鎮 > 上海浦東機場／桃園  
航班時間  
此為本行程預定的航班時間，實際航班以團體確認的航班編號與飛行時間為準。 
班機時間 
航空公司及航班編號 
起飛時間及城市 
抵達時間及城市 (跨日+1) 
2026/01/10(六)  
中國國際航空 CA186  
13:15  
台北(桃園)  


---

## 📝 3. PDF 轉換為 Markdown 格式

將提取的文字內容格式化為結構化的 Markdown 文件，方便 Claude 閱讀理解。

In [3]:
def convert_pdf_to_markdown(pdf_path, output_path):
    """將 PDF 轉換為結構化的 Markdown 文件"""
    doc = fitz.open(pdf_path)
    
    markdown_lines = []
    markdown_lines.append("# 哈爾濱旅行行程\n")
    markdown_lines.append("> 本文件由 PDF 自動轉換生成，供 Claude AI 和 Lovart.ai 參考使用\n")
    markdown_lines.append(f"> 轉換日期：{datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    markdown_lines.append("---\n")
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        
        if text.strip():
            markdown_lines.append(f"\n## 📄 第 {page_num + 1} 頁\n")
            
            # 處理文字內容
            lines = text.split('\n')
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                    
                # 嘗試識別標題（全大寫或特定模式）
                if re.match(r'^DAY\s*\d+', line, re.IGNORECASE):
                    markdown_lines.append(f"\n### 🗓️ {line}\n")
                elif re.match(r'^\d{1,2}[:/]\d{2}', line):
                    # 時間格式
                    markdown_lines.append(f"- ⏰ **{line}**")
                elif '景點' in line or '酒店' in line or '飯店' in line or '餐廳' in line:
                    markdown_lines.append(f"- 📍 {line}")
                elif '交通' in line or '飛機' in line or '火車' in line or '高鐵' in line:
                    markdown_lines.append(f"- 🚗 {line}")
                elif '美食' in line or '餐' in line or '吃' in line:
                    markdown_lines.append(f"- 🍜 {line}")
                else:
                    markdown_lines.append(line)
    
    doc.close()
    
    # 合併並寫入檔案
    markdown_content = '\n'.join(markdown_lines)
    
    # 確保目錄存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    return markdown_content

# 執行轉換
output_md_path = r"d:\Work\TestProject\產生旅行行程\docs\哈爾濱旅行行程.md"
markdown_content = convert_pdf_to_markdown(pdf_path, output_md_path)

print(f"✅ Markdown 文件已成功生成！")
print(f"📁 儲存位置: {output_md_path}")
print(f"📊 文件大小: {len(markdown_content)} 字元")
print("\n" + "=" * 60)
print("📝 Markdown 內容預覽 (前 3000 字元)")
print("=" * 60)
print(markdown_content[:3000])

✅ Markdown 文件已成功生成！
📁 儲存位置: d:\Work\TestProject\產生旅行行程\docs\哈爾濱旅行行程.md
📊 文件大小: 19654 字元

📝 Markdown 內容預覽 (前 3000 字元)
# 哈爾濱旅行行程

> 本文件由 PDF 自動轉換生成，供 Claude AI 和 Lovart.ai 參考使用

> 轉換日期：2025-12-07 21:40

---


## 📄 第 1 頁

團號：KRB011009CA6 東北冰雪奇緣～雪鄉蘑菇屋、冰雪大世界、魔界長白山、三晚五
- 📍 星飯店九日
含稅  無自費  無購物站  贈參團好禮
2026/01/10 台北出發 9 天｜訂金 15,000 /人
★ 哈爾濱國際冰雪節、冰雪大世界、雪鄉蘑菇屋
★ 魔界長白山(含霧淞飄流)、俄式風情伏爾加莊園
- 📍 ★ 季節限定：香格里拉冰宮火鍋、三晚品牌五星酒店
$48,500
- 📍 主要景點
- 📍 行程含27 個主要景點
D1 桃園／北京／長春
D2 長春 > 淨月潭森林公園(觀藍冰) > 地質宮外觀 > 長春文化廣場 > (車程約1.5HR) > 吉林 > 車遊臨江門大橋 > 天主教堂
廣場 > 吉林
D3 吉林 > 霧淞長廊 > (車程約4HR) > 長白山 > 長白山魔界景區(含霧淞漂流) > 長白山
D4 長白山 > 長白山北坡風景區(含環保車及倒站車)、天池、小天池、溫泉群、瀑布、綠淵潭 > (車程約2.5HR) > 敦化
D5 敦化 > (車程約4.5HR) > 鏡泊湖 > 鏡泊湖吊水樓冰瀑 > 鏡泊湖觀冬捕 > (車程約2HR) > 雪鄉 > 雪鄉夜景 > 雪鄉
D6 雪鄉 > 網紅打卡-雪韻大街 > 觀光棧道 > 雪鄉郵局 > 民俗博物館 > (車程約5H) > 伏爾加莊園
D7 伏爾加莊園(含高山雪圈每人一次) > 伏特加酒堡(品嚐伏特加酒+小點) > 彼得洛夫藝術宮 > (車程約1HR) > 哈爾濱 > 百
年濱洲鐵路橋 > 音樂公園網紅大雪人 > 中央大街 > 聖索菲亞教堂廣場 > 冰雪大世界 > (文化交流參訪) > 哈爾濱
D8 哈爾濱／上海 > EKA 天物 > 外觀浦東三大高樓 > 上海
D9 上海 > 新場古鎮 > 上海浦東機場／桃園
航班時間
此為本行程預定的航班時間，實際航班以團體確認的航

---

## 📊 4. 解析行程結構化資料

將旅遊行程解析為 JSON 結構化資料，包含日期、地點、活動等資訊。

In [4]:
# 根據 PDF 內容建立結構化行程資料
itinerary_data = {
    "trip_name": "東北冰雪奇緣～雪鄉蘑菇屋、冰雪大世界、魔界長白山",
    "duration": "9天",
    "departure_date": "2026/01/10",
    "return_date": "2026/01/18",
    "price": "NT$ 48,500",
    "highlights": [
        "哈爾濱國際冰雪節",
        "冰雪大世界",
        "雪鄉蘑菇屋",
        "魔界長白山(含霧淞漂流)",
        "俄式風情伏爾加莊園",
        "香格里拉冰宮火鍋",
        "三晚品牌五星酒店"
    ],
    "days": [
        {
            "day": 1,
            "date": "1/10",
            "title": "桃園／北京／長春",
            "locations": ["桃園", "北京", "長春"],
            "activities": ["搭機飛往北京轉機", "抵達長春"],
            "hotel": "長春高新益田福朋喜來登酒店(5星)",
            "meals": {"晚": "簡單宵夜點心"}
        },
        {
            "day": 2,
            "date": "1/11",
            "title": "長春 > 淨月潭 > 吉林",
            "locations": ["長春", "淨月潭森林公園", "地質宮", "長春文化廣場", "吉林", "臨江門大橋", "天主教堂廣場"],
            "activities": ["淨月潭森林公園(觀藍冰)", "地質宮外觀", "長春文化廣場", "車遊臨江門大橋", "天主教堂廣場"],
            "hotel": "吉林紫光苑華美達酒店(準5星)",
            "meals": {"早": "酒店內用", "午": "中式合菜", "晚": "烏拉滿族火鍋"}
        },
        {
            "day": 3,
            "date": "1/12",
            "title": "吉林 > 霧淞長廊 > 長白山魔界",
            "locations": ["吉林", "霧淞長廊", "長白山", "魔界景區"],
            "activities": ["霧淞長廊", "長白山魔界景區(含霧淞漂流)"],
            "hotel": "長白山王朝聖地溫泉酒店(準5星)",
            "meals": {"早": "酒店內用", "午": "朝鮮風味", "晚": "長白山珍宴"}
        },
        {
            "day": 4,
            "date": "1/13",
            "title": "長白山北坡風景區",
            "locations": ["長白山北坡", "天池", "小天池", "溫泉群", "瀑布", "綠淵潭", "敦化"],
            "activities": ["長白山北坡風景區", "天池", "小天池", "溫泉群", "長白瀑布", "綠淵潭"],
            "hotel": "敦化泰格酒店(4星)",
            "meals": {"早": "酒店內用", "午": "中式合菜", "晚": "烤肉風味"}
        },
        {
            "day": 5,
            "date": "1/14",
            "title": "鏡泊湖 > 雪鄉",
            "locations": ["敦化", "鏡泊湖", "吊水樓冰瀑", "雪鄉"],
            "activities": ["鏡泊湖吊水樓冰瀑", "鏡泊湖觀冬捕", "雪鄉夜景"],
            "hotel": "雪鄉內民宿(獨立衛浴)",
            "meals": {"早": "酒店內用", "午": "魚宴風味", "晚": "農家宴"}
        },
        {
            "day": 6,
            "date": "1/15",
            "title": "雪鄉 > 伏爾加莊園",
            "locations": ["雪鄉", "雪韻大街", "觀光棧道", "雪鄉郵局", "民俗博物館", "伏爾加莊園"],
            "activities": ["網紅打卡-雪韻大街", "觀光棧道", "雪鄉郵局(贈送明信片)", "民俗博物館"],
            "hotel": "伏爾加莊園賓館(4星)",
            "meals": {"早": "酒店內用", "午": "中式合菜", "晚": "莊園俄式西餐"}
        },
        {
            "day": 7,
            "date": "1/16",
            "title": "伏爾加莊園 > 哈爾濱",
            "locations": ["伏爾加莊園", "哈爾濱", "百年濱洲鐵路橋", "音樂公園", "中央大街", "聖索菲亞教堂", "冰雪大世界"],
            "activities": ["伏爾加莊園(含高山雪圈)", "伏特加酒堡", "彼得洛夫藝術宮", "百年濱洲鐵路橋", "音樂公園網紅大雪人", "中央大街", "聖索菲亞教堂廣場", "冰雪大世界"],
            "hotel": "哈爾濱松北融創皇冠假日酒店(5星)",
            "meals": {"早": "酒店內用", "午": "中式合菜", "晚": "香格里拉冰宮火鍋"}
        },
        {
            "day": 8,
            "date": "1/17",
            "title": "哈爾濱／上海",
            "locations": ["哈爾濱", "上海", "EKA天物", "浦東三大高樓"],
            "activities": ["搭機飛往上海", "EKA天物", "外觀浦東三大高樓"],
            "hotel": "上海中優城市萬豪酒店(5星)",
            "meals": {"早": "酒店內用", "午": "發餐費", "晚": "上海本幫菜"}
        },
        {
            "day": 9,
            "date": "1/18",
            "title": "上海 > 桃園",
            "locations": ["上海", "新場古鎮", "桃園"],
            "activities": ["新場古鎮", "搭機返回台北"],
            "hotel": "溫暖的家",
            "meals": {"早": "酒店內用", "午": "機上簡餐+生煎包"}
        }
    ]
}

# 儲存為 JSON 檔案
json_output_path = r"d:\Work\TestProject\產生旅行行程\docs\itinerary_data.json"
with open(json_output_path, 'w', encoding='utf-8') as f:
    json.dump(itinerary_data, f, ensure_ascii=False, indent=2)

print(f"✅ 結構化資料已儲存至: {json_output_path}")
print(f"\n📋 行程概覽:")
print(f"   行程名稱: {itinerary_data['trip_name']}")
print(f"   出發日期: {itinerary_data['departure_date']}")
print(f"   行程天數: {itinerary_data['duration']}")
print(f"   行程價格: {itinerary_data['price']}")
print(f"\n🌟 行程亮點:")
for highlight in itinerary_data['highlights']:
    print(f"   • {highlight}")

✅ 結構化資料已儲存至: d:\Work\TestProject\產生旅行行程\docs\itinerary_data.json

📋 行程概覽:
   行程名稱: 東北冰雪奇緣～雪鄉蘑菇屋、冰雪大世界、魔界長白山
   出發日期: 2026/01/10
   行程天數: 9天
   行程價格: NT$ 48,500

🌟 行程亮點:
   • 哈爾濱國際冰雪節
   • 冰雪大世界
   • 雪鄉蘑菇屋
   • 魔界長白山(含霧淞漂流)
   • 俄式風情伏爾加莊園
   • 香格里拉冰宮火鍋
   • 三晚品牌五星酒店


---

## 🎨 5. 產生 Lovart.ai 提示詞

根據行程資料自動產生 Lovart.ai 的圖片生成提示詞，用於製作旅遊行程視覺圖。

In [5]:
# 產生 Lovart.ai 提示詞
def generate_lovart_prompts(itinerary):
    """根據行程資料產生 Lovart.ai 圖片生成提示詞"""
    
    prompts = {
        "main_poster": f"""
🎨 【主海報提示詞】
Create a stunning travel itinerary infographic poster for "Northeast China Winter Wonderland Tour"

Style: Modern minimalist infographic, winter theme with ice blue and warm orange accents
Elements:
- Title: "{itinerary['trip_name']}"
- Duration: {itinerary['duration']}
- Date: {itinerary['departure_date']} - {itinerary['return_date']}
- Background: Snowy landscape with ice sculptures and traditional Chinese architecture
- Key visuals: Ice castle, snow-covered village (Snow Town), frozen lake, hot springs
- Color palette: Ice blue (#87CEEB), Snow white (#FFFAFA), Warm orange (#FFA500), Deep navy (#1A1A2E)
- Include decorative snowflakes and Chinese lanterns

Layout: Vertical poster format, A3 size ratio
""",
        
        "timeline": f"""
🗓️ 【時間軸圖提示詞】
Create a horizontal timeline infographic showing a 9-day winter travel itinerary in Northeast China

Style: Clean modern infographic with winter illustrations
Days to show:
Day 1: Taipei → Beijing → Changchun (airplane icon)
Day 2: Changchun → Jingyuetan Forest Park → Jilin (snowflake, blue ice)
Day 3: Jilin → Rime Corridor → Changbai Mountain Magic World (frost trees, mist)
Day 4: Changbai Mountain North Slope → Tianchi Lake → Dunhua (mountain, crater lake)
Day 5: Jingpo Lake Ice Waterfall → Snow Town Night (frozen waterfall, lanterns)
Day 6: Snow Town → Volga Manor (mushroom houses, Russian architecture)
Day 7: Volga Manor → Harbin → Ice and Snow World (ice sculptures, lights)
Day 8: Harbin → Shanghai (modern city)
Day 9: Shanghai → Return Home (ancient town)

Design elements: Dotted line connecting each day, small icons for each location
Color scheme: Winter blues, whites, with red lantern accents
""",

        "snow_town": """
❄️ 【雪鄉蘑菇屋插圖提示詞】
Create a magical illustration of Snow Town (雪鄉) in Heilongjiang Province, China

Scene: Traditional wooden houses covered with thick mushroom-shaped snow caps
Time: Evening/Night with warm orange lantern glow
Elements:
- Thick snow accumulated on rooftops forming natural mushroom shapes
- Red Chinese lanterns hanging along the streets
- Warm light glowing from windows
- Snow-covered pine trees in background
- Smoke rising from chimneys
- Clear starry winter sky
- Footprints in fresh snow
Style: Whimsical, cozy, fairy-tale like atmosphere
Mood: Warm and inviting despite the cold winter setting
""",

        "ice_world": """
🏰 【冰雪大世界插圖提示詞】
Create a spectacular illustration of Harbin Ice and Snow World (哈爾濱冰雪大世界)

Scene: Massive ice sculptures illuminated at night
Elements:
- Giant ice castles and towers made from crystal-clear ice blocks
- Colorful LED lights embedded within ice structures (blues, purples, greens, reds)
- Ice slides and winter activities
- Crowds of people enjoying the festival
- Traditional Russian-style ice architecture
- Frozen sculptures of dragons, temples, and famous landmarks
- Starry night sky with northern atmosphere
Style: Magical, grandiose, festival atmosphere
Color palette: Neon colors against white ice and dark sky
""",

        "changbai_mountain": """
🏔️ 【長白山天池插圖提示詞】
Create a breathtaking illustration of Changbai Mountain Tianchi (長白山天池) in winter

Scene: The famous crater lake surrounded by snow-covered peaks
Elements:
- Pristine blue/green crater lake (Heaven Lake)
- Snow and ice covering the surrounding volcanic peaks
- Wisps of steam rising from hot springs
- Clear winter sky with distant clouds
- Frost-covered trees on the mountain slopes
- Dramatic volcanic landscape
- Possible northern lights in sky
Style: Majestic, serene, natural wonder
Mood: Awe-inspiring, peaceful, spiritual
""",

        "map_style": """
🗺️ 【行程地圖插圖提示詞】
Create an illustrated travel map showing Northeast China winter tour route

Locations to mark:
1. Taipei (start) - Taiwan island
2. Beijing (transit)
3. Changchun - City icon
4. Jilin - Rime trees
5. Changbai Mountain - Mountain with lake
6. Dunhua - Train station
7. Jingpo Lake - Frozen waterfall
8. Snow Town - Mushroom houses
9. Volga Manor - Russian architecture
10. Harbin - Ice sculptures
11. Shanghai - Modern skyline
12. Return to Taipei

Style: Illustrated vintage travel map with decorative elements
Include: Dotted route line, small icons for each destination, compass rose, decorative border
Color scheme: Aged paper background, blue and white for winter themes, red accents
"""
    }
    
    return prompts

# 產生提示詞
prompts = generate_lovart_prompts(itinerary_data)

# 儲存提示詞到檔案
prompts_output_path = r"d:\Work\TestProject\產生旅行行程\docs\lovart_prompts.md"
with open(prompts_output_path, 'w', encoding='utf-8') as f:
    f.write("# Lovart.ai 旅遊行程圖片生成提示詞\n\n")
    f.write(f"> 行程: {itinerary_data['trip_name']}\n")
    f.write(f"> 日期: {itinerary_data['departure_date']} - {itinerary_data['return_date']}\n\n")
    f.write("---\n\n")
    
    for key, prompt in prompts.items():
        f.write(f"{prompt}\n\n---\n\n")

print(f"✅ Lovart.ai 提示詞已儲存至: {prompts_output_path}")
print("\n" + "=" * 60)
print("🎨 Lovart.ai 提示詞預覽")
print("=" * 60)
for key, prompt in list(prompts.items())[:2]:
    print(prompt[:500])
    print("...\n")

✅ Lovart.ai 提示詞已儲存至: d:\Work\TestProject\產生旅行行程\docs\lovart_prompts.md

🎨 Lovart.ai 提示詞預覽

🎨 【主海報提示詞】
Create a stunning travel itinerary infographic poster for "Northeast China Winter Wonderland Tour"

Style: Modern minimalist infographic, winter theme with ice blue and warm orange accents
Elements:
- Title: "東北冰雪奇緣～雪鄉蘑菇屋、冰雪大世界、魔界長白山"
- Duration: 9天
- Date: 2026/01/10 - 2026/01/18
- Background: Snowy landscape with ice sculptures and traditional Chinese architecture
- Key visuals: Ice castle, snow-covered village (Snow Town), frozen lake, hot springs
- Color palette: Ice blue (#87CEE
...


🗓️ 【時間軸圖提示詞】
Create a horizontal timeline infographic showing a 9-day winter travel itinerary in Northeast China

Style: Clean modern infographic with winter illustrations
Days to show:
Day 1: Taipei → Beijing → Changchun (airplane icon)
Day 2: Changchun → Jingyuetan Forest Park → Jilin (snowflake, blue ice)
Day 3: Jilin → Rime Corridor → Changbai Mountain Magic World (frost trees, mist)
Day 4: Chang

---

## 🌐 6. 建立互動式網頁 HTML

使用純 HTML/CSS/JavaScript 建立一個精美的互動式單頁網頁，無需後端伺服器即可運行。

In [6]:
# 產生互動式網頁 HTML
html_content = '''<!DOCTYPE html>
<html lang="zh-TW">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>東北冰雪奇緣 - 旅行行程</title>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        
        :root {
            --ice-blue: #87CEEB;
            --snow-white: #FFFAFA;
            --warm-orange: #FF8C42;
            --deep-navy: #1A1A2E;
            --frost-blue: #E8F4F8;
        }
        
        body {
            font-family: 'Microsoft JhengHei', 'Noto Sans TC', sans-serif;
            background: linear-gradient(135deg, var(--deep-navy) 0%, #16213E 50%, #0F3460 100%);
            min-height: 100vh;
            color: var(--snow-white);
        }
        
        /* 飄雪動畫 */
        .snowflakes { position: fixed; top: 0; left: 0; width: 100%; height: 100%; pointer-events: none; z-index: 1000; }
        .snowflake { position: absolute; color: #fff; opacity: 0.8; animation: fall linear infinite; }
        @keyframes fall {
            0% { transform: translateY(-10vh) rotate(0deg); opacity: 1; }
            100% { transform: translateY(110vh) rotate(360deg); opacity: 0.3; }
        }
        
        /* Header */
        header {
            background: linear-gradient(180deg, rgba(135,206,235,0.3) 0%, transparent 100%);
            padding: 40px 20px;
            text-align: center;
            position: relative;
        }
        
        .logo { font-size: 3em; margin-bottom: 10px; }
        
        h1 {
            font-size: 2.5em;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
            margin-bottom: 10px;
        }
        
        .trip-info {
            display: flex;
            justify-content: center;
            gap: 30px;
            flex-wrap: wrap;
            margin-top: 20px;
        }
        
        .trip-info-item {
            background: rgba(255,255,255,0.1);
            padding: 15px 25px;
            border-radius: 50px;
            backdrop-filter: blur(10px);
        }
        
        .trip-info-item i { color: var(--warm-orange); margin-right: 8px; }
        
        /* 行程亮點 */
        .highlights {
            padding: 40px 20px;
            max-width: 1200px;
            margin: 0 auto;
        }
        
        .section-title {
            text-align: center;
            font-size: 1.8em;
            margin-bottom: 30px;
            color: var(--ice-blue);
        }
        
        .highlight-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
        }
        
        .highlight-card {
            background: rgba(255,255,255,0.1);
            padding: 25px;
            border-radius: 15px;
            text-align: center;
            transition: transform 0.3s, background 0.3s;
            backdrop-filter: blur(10px);
        }
        
        .highlight-card:hover {
            transform: translateY(-10px);
            background: rgba(255,140,66,0.2);
        }
        
        .highlight-card i {
            font-size: 2.5em;
            color: var(--warm-orange);
            margin-bottom: 15px;
        }
        
        /* 時間軸 */
        .timeline {
            padding: 40px 20px;
            max-width: 1000px;
            margin: 0 auto;
        }
        
        .timeline-container {
            position: relative;
            padding-left: 40px;
        }
        
        .timeline-container::before {
            content: '';
            position: absolute;
            left: 15px;
            top: 0;
            bottom: 0;
            width: 3px;
            background: linear-gradient(180deg, var(--ice-blue), var(--warm-orange));
        }
        
        .day-card {
            background: rgba(255,255,255,0.08);
            border-radius: 15px;
            padding: 25px;
            margin-bottom: 25px;
            position: relative;
            transition: all 0.3s;
            cursor: pointer;
        }
        
        .day-card::before {
            content: '';
            position: absolute;
            left: -33px;
            top: 30px;
            width: 20px;
            height: 20px;
            background: var(--warm-orange);
            border-radius: 50%;
            border: 3px solid var(--deep-navy);
        }
        
        .day-card:hover {
            background: rgba(255,255,255,0.15);
            transform: translateX(10px);
        }
        
        .day-header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 15px;
        }
        
        .day-number {
            background: var(--warm-orange);
            padding: 8px 20px;
            border-radius: 20px;
            font-weight: bold;
        }
        
        .day-date { color: var(--ice-blue); }
        
        .day-title {
            font-size: 1.3em;
            margin-bottom: 15px;
            color: var(--snow-white);
        }
        
        .day-details { display: none; margin-top: 15px; }
        .day-card.active .day-details { display: block; }
        
        .activity-list {
            list-style: none;
            margin-top: 10px;
        }
        
        .activity-list li {
            padding: 8px 0;
            border-bottom: 1px solid rgba(255,255,255,0.1);
        }
        
        .activity-list li i {
            color: var(--ice-blue);
            margin-right: 10px;
            width: 20px;
        }
        
        .hotel-info, .meals-info {
            background: rgba(0,0,0,0.2);
            padding: 15px;
            border-radius: 10px;
            margin-top: 15px;
        }
        
        .hotel-info i, .meals-info i {
            color: var(--warm-orange);
            margin-right: 8px;
        }
        
        /* Footer */
        footer {
            text-align: center;
            padding: 40px 20px;
            background: rgba(0,0,0,0.3);
            margin-top: 40px;
        }
        
        .price-tag {
            font-size: 2.5em;
            color: var(--warm-orange);
            margin: 20px 0;
        }
        
        /* 響應式設計 */
        @media (max-width: 768px) {
            h1 { font-size: 1.8em; }
            .trip-info { gap: 15px; }
            .trip-info-item { padding: 10px 15px; font-size: 0.9em; }
        }
    </style>
</head>
<body>
    <!-- 飄雪效果 -->
    <div class="snowflakes" id="snowflakes"></div>
    
    <header>
        <div class="logo">❄️🏔️✈️</div>
        <h1>東北冰雪奇緣</h1>
        <p style="font-size: 1.2em; opacity: 0.9;">雪鄉蘑菇屋 • 冰雪大世界 • 魔界長白山</p>
        
        <div class="trip-info">
            <div class="trip-info-item">
                <i class="fas fa-calendar-alt"></i>
                <span>2026/01/10 - 01/18</span>
            </div>
            <div class="trip-info-item">
                <i class="fas fa-clock"></i>
                <span>9天8夜</span>
            </div>
            <div class="trip-info-item">
                <i class="fas fa-plane"></i>
                <span>台北出發</span>
            </div>
        </div>
    </header>
    
    <!-- 行程亮點 -->
    <section class="highlights">
        <h2 class="section-title">✨ 行程亮點</h2>
        <div class="highlight-grid">
            <div class="highlight-card">
                <i class="fas fa-snowflake"></i>
                <h3>哈爾濱冰雪大世界</h3>
                <p>世界最大冰雪主題樂園</p>
            </div>
            <div class="highlight-card">
                <i class="fas fa-home"></i>
                <h3>雪鄉蘑菇屋</h3>
                <p>童話般的雪中村莊</p>
            </div>
            <div class="highlight-card">
                <i class="fas fa-mountain"></i>
                <h3>長白山天池</h3>
                <p>神秘的火山口湖</p>
            </div>
            <div class="highlight-card">
                <i class="fas fa-magic"></i>
                <h3>魔界霧淞漂流</h3>
                <p>如夢似幻的霧淞仙境</p>
            </div>
            <div class="highlight-card">
                <i class="fas fa-chess-rook"></i>
                <h3>伏爾加莊園</h3>
                <p>俄式風情主題園區</p>
            </div>
            <div class="highlight-card">
                <i class="fas fa-fire"></i>
                <h3>香格里拉冰宮火鍋</h3>
                <p>冰與火的極致體驗</p>
            </div>
        </div>
    </section>
    
    <!-- 每日行程 -->
    <section class="timeline">
        <h2 class="section-title">🗓️ 每日行程</h2>
        <div class="timeline-container" id="timeline">
            <!-- 由 JavaScript 動態生成 -->
        </div>
    </section>
    
    <footer>
        <p style="font-size: 1.3em; margin-bottom: 10px;">團費（含稅）</p>
        <div class="price-tag">NT$ 48,500</div>
        <p style="opacity: 0.7;">無自費 • 無購物站 • 含稅</p>
        <p style="margin-top: 30px; opacity: 0.5;">東勢旅行社 | 業務代表：賴美惠 | 電話：02-25423355</p>
    </footer>
    
    <script>
        // 行程資料
        const days = [
            {
                day: 1, date: "1/10", title: "桃園／北京／長春",
                activities: ["搭乘中國國際航空飛往北京", "轉機前往長春", "抵達後入住酒店休息"],
                hotel: "長春高新益田福朋喜來登酒店(5星)",
                meals: { 晚: "簡單宵夜點心" }
            },
            {
                day: 2, date: "1/11", title: "長春 > 淨月潭 > 吉林",
                activities: ["淨月潭森林公園(觀藍冰)", "地質宮外觀", "長春文化廣場", "車遊臨江門大橋", "天主教堂廣場"],
                hotel: "吉林紫光苑華美達酒店(準5星)",
                meals: { 早: "酒店內用", 午: "中式合菜", 晚: "烏拉滿族火鍋" }
            },
            {
                day: 3, date: "1/12", title: "吉林 > 霧淞長廊 > 長白山魔界",
                activities: ["霧淞長廊", "長白山魔界景區", "霧淞漂流體驗"],
                hotel: "長白山王朝聖地溫泉酒店(準5星)",
                meals: { 早: "酒店內用", 午: "朝鮮風味", 晚: "長白山珍宴" }
            },
            {
                day: 4, date: "1/13", title: "長白山北坡風景區",
                activities: ["長白山北坡(含環保車及倒站車)", "天池", "小天池", "溫泉群", "長白瀑布", "綠淵潭"],
                hotel: "敦化泰格酒店(4星)",
                meals: { 早: "酒店內用", 午: "中式合菜", 晚: "烤肉風味" }
            },
            {
                day: 5, date: "1/14", title: "鏡泊湖 > 雪鄉",
                activities: ["鏡泊湖吊水樓冰瀑", "鏡泊湖觀冬捕", "雪鄉夜景"],
                hotel: "雪鄉內民宿(獨立衛浴)",
                meals: { 早: "酒店內用", 午: "魚宴風味", 晚: "農家宴" }
            },
            {
                day: 6, date: "1/15", title: "雪鄉 > 伏爾加莊園",
                activities: ["網紅打卡-雪韻大街", "觀光棧道", "雪鄉郵局(贈送明信片)", "民俗博物館"],
                hotel: "伏爾加莊園賓館(4星)",
                meals: { 早: "酒店內用", 午: "中式合菜", 晚: "莊園俄式西餐" }
            },
            {
                day: 7, date: "1/16", title: "伏爾加莊園 > 哈爾濱 > 冰雪大世界",
                activities: ["伏爾加莊園(含高山雪圈)", "伏特加酒堡", "彼得洛夫藝術宮", "百年濱洲鐵路橋", "音樂公園網紅大雪人", "中央大街", "聖索菲亞教堂廣場", "冰雪大世界"],
                hotel: "哈爾濱松北融創皇冠假日酒店(5星)",
                meals: { 早: "酒店內用", 午: "中式合菜", 晚: "香格里拉冰宮火鍋" }
            },
            {
                day: 8, date: "1/17", title: "哈爾濱／上海",
                activities: ["搭機飛往上海", "EKA天物創意園區", "外觀浦東三大高樓"],
                hotel: "上海中優城市萬豪酒店(5星)",
                meals: { 早: "酒店內用", 午: "發餐費", 晚: "上海本幫菜" }
            },
            {
                day: 9, date: "1/18", title: "上海 > 桃園",
                activities: ["新場古鎮", "搭機返回台北桃園機場"],
                hotel: "溫暖的家",
                meals: { 早: "酒店內用", 午: "機上簡餐+生煎包" }
            }
        ];
        
        // 產生時間軸
        const timelineContainer = document.getElementById('timeline');
        days.forEach(day => {
            const mealsHtml = Object.entries(day.meals || {})
                .map(([key, val]) => `${key}餐: ${val}`)
                .join(' | ');
            
            const card = document.createElement('div');
            card.className = 'day-card';
            card.innerHTML = `
                <div class="day-header">
                    <span class="day-number">DAY ${day.day}</span>
                    <span class="day-date">${day.date}</span>
                </div>
                <h3 class="day-title">${day.title}</h3>
                <p style="opacity: 0.7;">點擊查看詳細行程 ▼</p>
                <div class="day-details">
                    <ul class="activity-list">
                        ${day.activities.map(a => `<li><i class="fas fa-map-marker-alt"></i>${a}</li>`).join('')}
                    </ul>
                    <div class="hotel-info">
                        <i class="fas fa-hotel"></i>
                        <strong>住宿：</strong>${day.hotel}
                    </div>
                    <div class="meals-info">
                        <i class="fas fa-utensils"></i>
                        <strong>餐食：</strong>${mealsHtml}
                    </div>
                </div>
            `;
            card.addEventListener('click', () => card.classList.toggle('active'));
            timelineContainer.appendChild(card);
        });
        
        // 飄雪效果
        function createSnowflakes() {
            const container = document.getElementById('snowflakes');
            const snowflakeChars = ['❄', '❅', '❆', '✻', '✼'];
            
            for (let i = 0; i < 50; i++) {
                const snowflake = document.createElement('div');
                snowflake.className = 'snowflake';
                snowflake.innerHTML = snowflakeChars[Math.floor(Math.random() * snowflakeChars.length)];
                snowflake.style.left = Math.random() * 100 + 'vw';
                snowflake.style.fontSize = (Math.random() * 15 + 10) + 'px';
                snowflake.style.animationDuration = (Math.random() * 5 + 5) + 's';
                snowflake.style.animationDelay = (Math.random() * 10) + 's';
                container.appendChild(snowflake);
            }
        }
        createSnowflakes();
    </script>
</body>
</html>
'''

# 儲存 HTML 檔案
html_output_path = r"d:\Work\TestProject\產生旅行行程\docs\index.html"
with open(html_output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✅ 互動式網頁已儲存至: {html_output_path}")
print(f"\n📱 網頁功能:")
print("   • 飄雪動畫效果")
print("   • 行程亮點卡片展示")
print("   • 可展開/收合的每日行程時間軸")
print("   • 響應式設計（支援手機瀏覽）")
print("\n🔗 您可以直接用瀏覽器開啟 index.html 預覽效果")

✅ 互動式網頁已儲存至: d:\Work\TestProject\產生旅行行程\docs\index.html

📱 網頁功能:
   • 飄雪動畫效果
   • 行程亮點卡片展示
   • 可展開/收合的每日行程時間軸
   • 響應式設計（支援手機瀏覽）

🔗 您可以直接用瀏覽器開啟 index.html 預覽效果


---

## 🚀 7. 部署至免費雲端平台

以下提供多種免費部署方案，適合低流量、短期使用的旅行行程網站。

### 📋 方案比較

| 平台 | 免費額度 | 設定難度 | 適用情境 |
|------|----------|----------|----------|
| **GitHub Pages** | 無限制 | ⭐ 簡單 | 靜態網頁，永久免費 |
| **Vercel** | 100GB/月 | ⭐⭐ 中等 | 靜態/動態網站 |
| **Netlify** | 100GB/月 | ⭐⭐ 中等 | 靜態網站，表單功能 |
| **Cloudflare Pages** | 無限制 | ⭐⭐ 中等 | 全球 CDN，速度快 |

---

### 🎯 推薦方案：GitHub Pages（最簡單免費）

因為您的需求是「流量非常少，只有旅行期間使用」，**GitHub Pages** 是最佳選擇！

#### 📝 部署步驟教學：

**步驟 1：註冊 GitHub 帳號**
1. 前往 https://github.com
2. 點擊 "Sign up" 註冊帳號
3. 完成郵箱驗證

**步驟 2：建立新的 Repository**
1. 登入後，點擊右上角 "+" → "New repository"
2. Repository name 填入：`harbin-trip`（或任意名稱）
3. 選擇 "Public"
4. 勾選 "Add a README file"
5. 點擊 "Create repository"

**步驟 3：上傳網頁檔案**
1. 在 Repository 頁面，點擊 "Add file" → "Upload files"
2. 將 `docs/index.html` 拖曳上傳
3. 點擊 "Commit changes"

**步驟 4：啟用 GitHub Pages**
1. 進入 Repository → "Settings" → 左側選單 "Pages"
2. Source 選擇 "Deploy from a branch"
3. Branch 選擇 "main"，資料夾選擇 "/ (root)"
4. 點擊 "Save"

**步驟 5：取得網址**
- 等待約 1-2 分鐘後，頁面上會顯示：
- `Your site is live at https://你的帳號.github.io/harbin-trip/`

🎉 **完成！** 將此網址分享給旅伴即可！

---

### 🔄 備用方案：Vercel（自動化部署）

如果您熟悉 Git，Vercel 提供更自動化的體驗：

1. 前往 https://vercel.com
2. 使用 GitHub 帳號登入
3. 點擊 "New Project" → 選擇您的 GitHub Repository
4. Vercel 會自動偵測並部署
5. 取得免費網址：`https://harbin-trip.vercel.app`

---

### 🔄 備用方案：Netlify Drop（拖曳即部署）

最快速的方式，無需註冊：

1. 前往 https://app.netlify.com/drop
2. 直接將 `docs` 資料夾拖曳到網頁上
3. 等待幾秒鐘即可取得網址
4. **注意**：未註冊的部署會在一段時間後過期

---

In [7]:
# 產生部署用的 README 檔案
readme_content = '''# 🏔️ 東北冰雪奇緣 - 旅行行程網站

這是一個互動式旅行行程展示網站，展示 2026 年 1 月東北冰雪之旅的詳細行程。

## 🌟 行程亮點

- ❄️ 哈爾濱冰雪大世界
- 🏠 雪鄉蘑菇屋
- 🏔️ 長白山天池
- ✨ 魔界霧淞漂流
- 🏰 伏爾加莊園
- 🔥 香格里拉冰宮火鍋

## 📅 行程日期

2026/01/10 - 2026/01/18（9天8夜）

## 💰 團費

NT$ 48,500（含稅，無自費、無購物站）

## 🔗 線上預覽

部署後的網站網址：[待填入]

## 📁 檔案說明

- `index.html` - 主要網頁檔案
- `哈爾濱旅行行程.md` - Markdown 格式行程文件
- `itinerary_data.json` - 結構化行程資料
- `lovart_prompts.md` - Lovart.ai 圖片生成提示詞

## 🚀 部署方式

本網站為純靜態 HTML 網頁，可部署至：
- GitHub Pages
- Vercel
- Netlify
- Cloudflare Pages

---

*由 Claude AI 協助生成*
'''

readme_path = r"d:\Work\TestProject\產生旅行行程\docs\README.md"
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"✅ README.md 已儲存至: {readme_path}")

# 顯示已生成的所有檔案
print("\n" + "=" * 60)
print("📁 已生成的檔案清單")
print("=" * 60)

import os
docs_path = r"d:\Work\TestProject\產生旅行行程\docs"
for filename in os.listdir(docs_path):
    filepath = os.path.join(docs_path, filename)
    size = os.path.getsize(filepath)
    print(f"   📄 {filename} ({size:,} bytes)")

print("\n" + "=" * 60)
print("🎉 所有檔案已生成完畢！")
print("=" * 60)

✅ README.md 已儲存至: d:\Work\TestProject\產生旅行行程\docs\README.md

📁 已生成的檔案清單
   📄 index.html (15,747 bytes)
   📄 itinerary_data.json (5,624 bytes)
   📄 lovart_prompts.md (4,702 bytes)
   📄 README.md (961 bytes)
   📄 哈爾濱旅行行程.md (52,876 bytes)

🎉 所有檔案已生成完畢！
